In [ ]:
import numpy as np
import pandas as pd
from scipy.integrate import solve_ivp
from pathlib import Path

pd.set_option('display.max_columns', None)
%matplotlib inline

In [7]:
CONFIG = {
    "g": 9.81,
    "L1": 1.0,
    "L2": 1.0,
    "m1": 1.0,
    "m2": 1.0,
    "theta1_deg": 120,
    "theta2_deg": -10,
    "omega1_0": 0.0,
    "omega2_0": 0.0,
    "t_start": 0,
    "t_end": 50,
    "num_points": 5000,
    "output_path": "../data/raw/double_pendulum_simulation.csv"
}


In [8]:
def double_pendulum_derivatives(t, y, cfg):
    θ1, θ2, ω1, ω2 = y
    Δ = θ2 - θ1
    m1, m2, L1, L2, g = cfg["m1"], cfg["m2"], cfg["L1"], cfg["L2"], cfg["g"]

    denom = (2 * m1 + m2 - m2 * np.cos(2 * Δ))

    dω1_dt = (
        -g * (2 * m1 + m2) * np.sin(θ1)
        - m2 * g * np.sin(θ1 - 2 * θ2)
        - 2 * np.sin(Δ) * m2 * (ω2**2 * L2 + ω1**2 * L1 * np.cos(Δ))
    ) / (L1 * denom)

    dω2_dt = (
        2 * np.sin(Δ) * (
            ω1**2 * L1 * (m1 + m2)
            + g * (m1 + m2) * np.cos(θ1)
            + ω2**2 * L2 * m2 * np.cos(Δ)
        )
    ) / (L2 * denom)

    return [ω1, ω2, dω1_dt, dω2_dt]


In [9]:
def run_simulation(cfg):
    print("🔧 Running simulation...")
    
    θ1_0 = np.radians(cfg["theta1_deg"])
    θ2_0 = np.radians(cfg["theta2_deg"])
    ω1_0 = cfg["omega1_0"]
    ω2_0 = cfg["omega2_0"]
    initial_state = [θ1_0, θ2_0, ω1_0, ω2_0]

    t_eval = np.linspace(cfg["t_start"], cfg["t_end"], cfg["num_points"])

    sol = solve_ivp(
        fun=lambda t, y: double_pendulum_derivatives(t, y, cfg),
        t_span=(cfg["t_start"], cfg["t_end"]),
        y0=initial_state,
        t_eval=t_eval,
        method='RK45',
        rtol=1e-9,
        atol=1e-9
    )

    θ1, θ2, ω1, ω2 = sol.y
    t = sol.t

    x1 = cfg["L1"] * np.sin(θ1)
    y1 = -cfg["L1"] * np.cos(θ1)
    x2 = x1 + cfg["L2"] * np.sin(θ2)
    y2 = y1 - cfg["L2"] * np.cos(θ2)

    return pd.DataFrame({
        "time": t,
        "theta1": θ1,
        "theta2": θ2,
        "omega1": ω1,
        "omega2": ω2,
        "x1": x1,
        "y1": y1,
        "x2": x2,
        "y2": y2
    })


In [10]:
output_file = Path(CONFIG["output_path"])
output_file.parent.mkdir(parents=True, exist_ok=True)

if output_file.exists():
    print(f"📄 File found at {output_file}. Loading existing data...")
    df = pd.read_csv(output_file)
else:
    df = run_simulation(CONFIG)
    df.to_csv(output_file, index=False)
    print(f"✅ Data generated and saved to {output_file}")


📄 File found at ..\data\raw\double_pendulum_simulation.csv. Loading existing data...


In [11]:
def compute_energies(df, cfg):
    g = cfg["g"]
    m1 = cfg["m1"]
    m2 = cfg["m2"]
    L1 = cfg["L1"]
    L2 = cfg["L2"]

    θ1 = df["theta1"]
    θ2 = df["theta2"]
    ω1 = df["omega1"]
    ω2 = df["omega2"]

    KE1 = 0.5 * m1 * (L1**2) * ω1**2
    KE2 = 0.5 * m2 * (
        (L1**2 * ω1**2) +
        (L2**2 * ω2**2) +
        2 * L1 * L2 * ω1 * ω2 * np.cos(θ1 - θ2)
    )

    y1 = -L1 * np.cos(θ1)
    y2 = y1 - L2 * np.cos(θ2)

    PE1 = m1 * g * (y1 + L1)  
    PE2 = m2 * g * (y2 + L1 + L2)

    TME = KE1 + KE2 + PE1 + PE2

    df["KE"] = KE1 + KE2
    df["PE"] = PE1 + PE2
    df["TME"] = TME
    return df


In [12]:
df_processed = compute_energies(df.copy(), CONFIG)

processed_path = Path("../data/processed/double_pendulum_processed.csv")
processed_path.parent.mkdir(parents=True, exist_ok=True)
df_processed.to_csv(processed_path, index=False)

print(f"📁 Processed data saved to {processed_path}")


📁 Processed data saved to ..\data\processed\double_pendulum_processed.csv
